# Student Loan Risk with Deep Learning

In [34]:
# Imports
import pandas as pd
import tensorflow as tf
import sklearn as skl
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path


---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [35]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [36]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [37]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [38]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y.head()

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [39]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop("credit_ranking", axis=1)

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [40]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [41]:
# Create a StandardScaler instance
X_scaler = skl.preprocessing.StandardScaler()


# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [42]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)
# Review the number of features
print(len(X.columns))

11


In [43]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  6

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 4

# Define the number of neurons in the output layer
output_layer = 1

In [44]:
# Create the Sequential model instance
nn = tf.keras.models.Sequential()

# Add the first hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_nodes, activation="relu"))

# Add the second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(tf.keras.layers.Dense(units=output_layer, activation="sigmoid"))

In [45]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 6)                 72        
                                                                 
 dense_7 (Dense)             (None, 4)                 28        
                                                                 
 dense_8 (Dense)             (None, 1)                 5         
                                                                 
Total params: 105 (420.00 Byte)
Trainable params: 105 (420.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [46]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [47]:
# Fit the model using 50 epochs and the training data
fit_model = nn.fit(X_train_scaled,y_train,epochs=50)

Epoch 1/50
38/38 [==============================] - 1s 2ms/step - loss: 0.6701 - accuracy: 0.5738
Epoch 2/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6493 - accuracy: 0.6305
Epoch 3/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6272 - accuracy: 0.6764
Epoch 4/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6040 - accuracy: 0.7139
Epoch 5/50
38/38 [==============================] - 0s 3ms/step - loss: 0.5838 - accuracy: 0.7098
Epoch 6/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5660 - accuracy: 0.7214
Epoch 7/50
38/38 [==============================] - 0s 3ms/step - loss: 0.5528 - accuracy: 0.7214
Epoch 8/50
38/38 [==============================] - 0s 3ms/step - loss: 0.5434 - accuracy: 0.7273
Epoch 9/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5360 - accuracy: 0.7289
Epoch 10/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5309 - accuracy: 0.7331
Epoch 11/50
38/38 [

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [48]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 0.5280 - accuracy: 0.7300 - 137ms/epoch - 11ms/step
Loss: 0.5280115604400635, Accuracy: 0.7300000190734863


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [49]:
##Set up saved models path
Path("saved_models").mkdir(parents=True, exist_ok=True)

# Set the model's file path
file_path = Path("saved_models/student_loans.keras")

# Export your model to a keras file
nn.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [50]:
# Set the model's file path
file_path = Path("saved_models/student_loans.keras")

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [51]:
# Make predictions with the test data
predictions = nn_imported.predict(X_test_scaled,verbose=2)

# Display a sample of the predictions
predictions[0:5]

13/13 - 0s - 69ms/epoch - 5ms/step


array([[0.5463359 ],
       [0.28645745],
       [0.7902565 ],
       [0.778509  ],
       [0.9794577 ]], dtype=float32)

In [52]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(columns=["predictions"], data=predictions)
predictions_df["predictions"] = round(predictions_df["predictions"],0)
predictions_df

,predictions
0,1.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [53]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["predictions"].values))

              precision    recall  f1-score   support

           0       0.71      0.72      0.71       188
           1       0.75      0.74      0.74       212

    accuracy                           0.73       400
   macro avg       0.73      0.73      0.73       400
weighted avg       0.73      0.73      0.73       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. If I were building my own studen loan recommendation system I would include many of the fields in the provide data including; gpa ranking, alumni success, study major_code, time to completion, finance workshop score, and credit ranking.  There are many more I would collect in addition:    
**Age:**  Many students applying for loans are young perhapse 18-24, but there are others that may have been working for sometime or are going for graduate degrees.
**Credit Score:** While not all students will have a relevant credit score, some may, and if available would be a good way to determine the students capability and history of making payments on time.    
**Current Salary:** Existing income would be very helpful in understanding credit worthiness.  
**Time in current role/field:**A student whoes been employed in a field for 5+ years is more than likely to be able to maintain an income in the field.  A short term might jsut indicate part time college work.  
**Previous Yr Adjusted Gross Income:**  Again, while not all students will have a previous income if their just comeing out of high school, some may.  
**Existing Student Loan Balance:** Knowing the existing level of student loan debt will provid understanding  on applicants ability to meet payment terms
   
**Average pay of major 5 years post degreee:** Combining the average expected income post degree with the existing loan balance figure will again provide more insights on an applicatns ability to meet payment terms.  
**Cosigner on Loan(Y/N):** A co-signer on a loan provides an additional revenue stream to ensure payment terms are met.  
**Cosigner Credit Score:** For siimilar reasons as you would want to know the applicants credit score, this will give a data point on payment history.

**Cosigner previous yr adjusted Gross Income:**  Verifies the cosigners ability to afford payment terms.

These additional data points gathered over time, and matched with a second database capturing payments made post degree by previous students approved for a loan should give you a very good idea of credit worthiness.  
  
  
**2.** Based on the data I chose to use in the recommendation section, my model would use collabortive filtering as this method recommends items based on similarity measures between users.  The data points provided by the loan applicant would be evaulated and compared to the payment history of previous/current approved applicants.  Recommendations would be made using the assumption that similarities in application data would result in similar payback ability.


**3.** Two real world challenges that I would take into consideration while buidling this recommendation system would be the relevance of a credit score and ability to provide a co-signer.  
The first challenge, relevance of a credit score, could be hanled by applying a weight to that metric prior to running the model.  Credit score could be weighted heavier for those applicants who are over a certain age, and therefore have had more time to actually earn the score as oppose to those whove just started building a payment history.  
The second challenge, the ability to provide a co-signer is a bit trickier.  I imagine most loans with a co-signer may float to the top of the recommedned to receive a loan list.  However, an individuals ability to payback their own loan isn't dependent on them knowing someone who is credit worthy. I can see two options to tackle this problem. Prior to the model less weighting could be applied to the presence of a co-applicant and the associated fields.  The second option might be to create two models, one to handel loans with a co-signer and one for those without and setting approval standards for each group seperately.  I believe this second methodology might be the most appropriate in the end.

